# Embedding sample notebook

The following notebook demonstartes how to embed your Power BI content in a Jupyter notebook and interact with it.
To run the demo, you will need to obtain your report id and group (workspace) id from the URL of your report.

In [3]:
from powerbiclient.authentication import DeviceCodeLoginAuthentication
from powerbiclient import Report, models
from io import StringIO
from ipywidgets import interact
import requests
import pandas as pd
import matplotlib

### Authenticate to Power BI


#### The following authentication methods can be used to authenticate the user and get the access token:

- [Device flow authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_by_device_flow)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import DeviceCodeLoginAuthentication
  device_auth = DeviceCodeLoginAuthentication()
    ```
- [Interactive authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_interactive)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import InteractiveLoginAuthentication
  interactive_auth = InteractiveLoginAuthentication()    
    ```

Note: Either use auth object created or access token acquired above while creating the report instance


#### Run the cell below to obtain the auth object using the Device Flow Authentication method for the demo notebook

In [4]:
# Auth object for demo
auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E6JA3DRJ7 to authenticate.


RuntimeError: Authentication failed. Try again.
Details: AADSTS70016: OAuth 2.0 device flow error. Authorization is pending. Continue polling. Trace ID: 8eb7d058-555c-4695-aa3c-61c338460f00 Correlation ID: 57345ce2-e85d-4657-a396-e91f1189d7da Timestamp: 2025-09-10 17:19:18Z

### Create an instance of Power BI Report

- Below snippets are the values for optional view_mode parameter
``` py 
view_mode=models.EmbedMode.VIEW.value
view_mode=models.EmbedMode.EDIT.value
view_mode=models.EmbedMode.CREATE.value
``` 
Default value for the parameter is 
``` py 
view_mode=models.EmbedMode.VIEW.value 
```

- Below snippets are the values for the [permissions](https://docs.microsoft.com/en-us/javascript/api/overview/powerbi/configure-report-settings#open-in-edit-mode) parameter for embedding the report in EDIT mode
``` py 
permissions=models.Permissions.READWRITE.value
permissions=models.Permissions.COPY.value
permissions=models.Permissions.ALL.value
``` 

#### Render report in Create mode
To create a new report in embed mode, the following additional parameters need to be passed
``` py
dataset_id='POWER_BI_DATASET_ID'
view_mode=models.EmbedMode.CREATE.value
```

#### Replace the group_id and report_id parameters with your own report ID and group ID and run the cell below to create a Power BI report instance:

In [ ]:
group_id = "ENTER YOUR GROUP ID"
report_id = "ENTER YOUR REPORT ID"
report = Report(group_id=group_id, report_id=report_id, auth=auth)

### Power BI event handlers

#### Set report loaded event handler

In [ ]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

#### Set report rendered event handler

In [ ]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

#### Load the report in the output cell

In [ ]:
# Render report
report

#### Change dimensions of embedded report

In [ ]:
# Change dimensions of embedded report
report.set_size(600, 900)

### Export data from a visual of the embedded report

#### Get the active page

In [ ]:
def get_active_page(self):
    
    # Get list of pages
    pages = self.get_pages()
    active_page = {}
    for page in pages:
        
        # Get active page
        if page['isActive'] == True:
            active_page = page
            break
    return active_page

#### Get the visual whose data is to be exported

Make sure that the active page on the report you're embedding has a visual of type 'clustered column chart', or change to a different visual type below.

In [ ]:
active_page = get_active_page(report)
active_page_name = active_page['name']

# Get list of visuals on active page
visuals = report.visuals_on_page(active_page_name)

# Get visual whose type is 'clusteredColumnChart'
visual = next(filter(lambda visual: visual['type'] == 'clusteredColumnChart', visuals))
visual_name = visual['name']

#### Export summarized data from a visual of the embedded report

In [ ]:
summarized_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 20)

print(summarized_exported_data)

#### Export underlying data from a visual of the embedded report

In [ ]:
underlying_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 5, export_data_type = models.ExportDataType.UNDERLYING.value)

print(underlying_exported_data)

### Convert exported visual data into CSV and load into pandas dataframe

In [ ]:
data = StringIO(summarized_exported_data)

df = pd.read_csv(data, sep=",")
df.head()

### Visualize the data

Adjust the plot parameters below to fit your report

In [ ]:
df.plot.pie(y="Total Units", labels=df.loc[:,'isVanArsdel'].values.tolist(), figsize=(10,15))

### Filter the report by region using iPyWidget

Adjust the filter below to fit your report

In [ ]:
def filter_report(Region):
    region_filter = {
        '$schema': "http://powerbi.com/product/schema#basic",
        'target': {
            'table': "Geo",
            'column': "Region"
        },
        'operator': "In",
        'values': [Region]
    }
    report.remove_filters()
    report.update_filters([region_filter])

In [ ]:
report

In [ ]:
interact(filter_report, Region=['Central','East', 'West']);

### Get filters applied on the report

In [ ]:
report.get_filters()

### Get report bookmarks

In [ ]:
# Get list of bookmarks
bookmarks_list = report.get_bookmarks()

# Convert the list into a dataframe
bookmarks_dataframe = pd.DataFrame(bookmarks_list)

# Rename columns
bookmarks_dataframe.rename(columns={'name':'Name', 'displayName':'Display Name', 'state':'State'}, inplace=True)
bookmarks_dataframe

### Apply report bookmark

In [ ]:
def apply_bookmark(BookmarkDisplayName):
    
    # Get bookmark name corresponding to the display name provided by the user
    bookmark_name = bookmarks_dataframe[bookmarks_dataframe['Display Name'] == BookmarkDisplayName].Name.values[0]
    report.set_bookmark(bookmark_name)

### Apply bookmark using ipywidget

In [ ]:
report

In [ ]:
interact(apply_bookmark, BookmarkDisplayName=bookmarks_dataframe['Display Name'])